In [1]:
#import library and module
import pandas as pd
from resume_screening import resparser, match

c:\Users\Acer\anaconda3\envs\resume-parser\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
from nltk.corpus import stopwords
stopw  = set(stopwords.words('english'))

In [3]:
#read indeed job vacancy data
job = pd.read_csv('indeed_data.csv')
job['test'] = job['description'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word)>2 and word not in (stopw)]))

In [4]:
df = job.drop_duplicates(subset='test').reset_index(drop=True)
df.head()

title            company salary  \
0     Data Scientist (Analytics)               Grab   None   
1                 Data Scientist           Sayurbox   None   
2  Senior Data Scientist - GoPay             GO-JEK   None   
3                 Data Scientist  Allianz Indonesia   None   
4          Senior Data Scientist           Sayurbox   None   

                                         description  \
0  Job Description: Life at Grab At Grab, every G...   
1  Job Description Be a part of Sayurbox data tea...   
2  Location Jakarta Work Type Permanent Applicati...   
3  Bachelor Degree with field of study mathematic...   
4  Job Description: Build Sayurbox data team acco...   

                                                link  \
0  https://id.indeed.com/rc/clk?jk=acd6479660d3fc...   
1  https://id.indeed.com/rc/clk?jk=40482050a5e4d8...   
2  https://id.indeed.com/rc/clk?jk=60463a42bb19a6...   
3  https://id.indeed.com/rc/clk?jk=835f2c980867fe...   
4  https://id.indeed.com/rc/clk?jk=da6afcc2ee471d...   

                                                test  
0  Job Description: Life Grab Grab, every Grabber...  
1  Job Description part Sayurbox data team target...  
2  Location Jakarta Work Type Permanent Applicati...  
3  Bachelor Degree field study mathematic, inform...  
4  Job Description: Build Sayurbox data team acco...

In [5]:
test = (df['test'].values.astype('U'))

In [6]:
print("There are {} job vacancy list scrapped from id.indeed.com".format(df.shape[0]))

There are 567 job vacancy list scrapped from id.indeed.com


# Input CV & Preprocessing

In [186]:
resumeList = []
similarityPercentage =[]

In [295]:
#input the resume or CV file
filled = input()
resumeList.append(filled)
skills = resparser.skill("instance/resume_files/{}".format(filled))

In [296]:
skills.append(match.preprocessing(skills[0]))
del skills[0]

print("SKILLS LIST:\n",skills)

SKILLS LIST:
 ['figma instagram organization investors feeds usability testing higher education banners mastery moc teaching social media user interface design space linkedin export google interfaces app wireframing internships ux online training mobile interview building materials ux design prototype design media design machine learning options promotional manuals ug content design dsc usability web ued mobile applications user experience design learning visual design java experience design illustration gsm web design developers sketching user experience digital graphic design testing']


# Model KNN - Minkowski Metric

In [297]:
#Train the KNN model using Minkowski Metric
matches = match.knearestNeighbors(skills, test)
matches = pd.DataFrame(matches, columns=['Match confidence'])

In [298]:
#Job Vacancy Recommendation
result_minkowski = df[['title','company','salary']].copy()
result_minkowski['match'] = matches['Match confidence'].copy()
result_minkowski = result_minkowski.sort_values('match').reset_index(drop=True).head(20)
result_minkowski

title  \
0                                 UX Designer   
1              UX/UI Designer (Ref. No. 2156)   
2                            UX / UI Designer   
3                       Senior UI/UX Designer   
4                    UI Designer - Internship   
5                             UI/ UX Designer   
6                       Senior UI/UX Designer   
7                            UI / UX Designer   
8                       Senior UI/UX Designer   
9                     UI/UX Designer (Senior)   
10                             UI/UX Designer   
11                      Senior UI/UX Designer   
12                   Associate UI/UX Designer   
13                             UI/UX Designer   
14                      Senior UI/UX Designer   
15                             UI/UX Designer   
16                             UI/UX DESIGNER   
17  B2C Support - Creative and UI/UX Designer   
18                             UI/UX Designer   
19                                  UX Writer   

                                 company  \
0   PT HZN Teknologi Indonesia (Horizon)   
1        Global Landscapes Forum (CIFOR)   
2                               Startups   
3                               Tokenomy   
4   PT HZN Teknologi Indonesia (Horizon)   
5                             Tinkerlust   
6                            Cermati.com   
7               PT Mobile Sarana Sentosa   
8                       Whello Indonesia   
9                               Pensieve   
10     Mekari (PT. Mid Solusi Nusantara)   
11               PT Nusa Satu Inti Artha   
12                               Devhaus   
13                         1001malam.com   
14         DOKU, PT NUSA SATU INTI ARTHA   
15                             sribu.com   
16                           Dipstrategy   
17                PT Asuransi Jiwa Astra   
18                                 Aleph   
19                                  NOBI   

                                     salary  match  
0                                      None   0.54  
1                                      None   0.56  
2                                      None   0.57  
3                                      None   0.57  
4                                      None   0.57  
5                                      None   0.57  
6                                      None   0.60  
7                                      None   0.61  
8                                      None   0.62  
9                                      None   0.62  
10                                     None   0.62  
11                                     None   0.62  
12                                     None   0.62  
13                                     None   0.64  
14                                     None   0.64  
15  Rp. 6.000.000 - Rp. 8.000.000 per bulan   0.65  
16                                     None   0.65  
17                                     None   0.65  
18                                     None   0.65  
19                                     None   0.65

# Model Cosine Similarity

In [299]:
df['clean'] = df['test'].apply(match.preprocessing)
jobdesc = (df['clean'].values.astype('U'))

In [300]:
count_matrix = match.vectorizing(skills[0], jobdesc)
matchPercentage = match.coSim(count_matrix)
matchPercentage = pd.DataFrame(matchPercentage, columns=['Skills Match'])

In [301]:
#Job Vacancy Recommendations
result_cosine = df[['title','company','salary']]
result_cosine = result_cosine.join(matchPercentage)
result_cosine = result_cosine.sort_values('Skills Match', ascending=False).reset_index(drop=True).head(20)
result_cosine

title                               company salary  \
0   UI Designer - Internship  PT HZN Teknologi Indonesia (Horizon)   None   
1                UX Designer  PT HZN Teknologi Indonesia (Horizon)   None   
2      Senior UI/UX Designer                      Whello Indonesia   None   
3      Senior UI/UX Designer         DOKU, PT NUSA SATU INTI ARTHA   None   
4           UI / UX Designer              PT Mobile Sarana Sentosa   None   
5             UI/UX DESIGNER                           Dipstrategy   None   
6            UI/ UX Designer                            Tinkerlust   None   
7      Senior UI/UX Designer               PT Nusa Satu Inti Artha   None   
8             UI/UX Designer                              Dekoruma   None   
9             UI/UX Designer   Nomura Research Institute Indonesia   None   
10     Senior UI/UX Designer                              Tokenomy   None   
11            UI/UX Designer              NTT INDONESIA TECHNOLOGY   None   
12            UI/UX Designer                                 Aleph   None   
13            UI/UX Designer           Tera Logistik Internasional   None   
14                 UX Writer                                  NOBI   None   
15          UX / UI Designer                              Startups   None   
16  UI/UX Designer (Officer)          PT Majoo Teknologi Indonesia   None   
17            UI/UX Designer     Mekari (PT. Mid Solusi Nusantara)   None   
18            UI/UX Designer                  Bali Staff Solutions   None   
19            UI/UX Designer                         1001malam.com   None   

    Skills Match  
0          68.50  
1          68.12  
2          56.29  
3          55.31  
4          54.49  
5          54.38  
6          52.51  
7          52.49  
8          50.85  
9          48.13  
10         47.80  
11         47.71  
12         47.67  
13         47.17  
14         46.56  
15         46.14  
16         45.76  
17         45.58  
18         45.06  
19         42.74

# Comparing between Minkowski Metric vs Cosine Similarity

In [302]:
data = [result_minkowski[['title','company']], result_cosine[['title','company']]]
headers = ['Minkowski','Cosine Similarity']
df_compare = pd.concat(data, axis = 1, keys = headers)
df_compare

Minkowski  \
                                        title   
0                                 UX Designer   
1              UX/UI Designer (Ref. No. 2156)   
2                            UX / UI Designer   
3                       Senior UI/UX Designer   
4                    UI Designer - Internship   
5                             UI/ UX Designer   
6                       Senior UI/UX Designer   
7                            UI / UX Designer   
8                       Senior UI/UX Designer   
9                     UI/UX Designer (Senior)   
10                             UI/UX Designer   
11                      Senior UI/UX Designer   
12                   Associate UI/UX Designer   
13                             UI/UX Designer   
14                      Senior UI/UX Designer   
15                             UI/UX Designer   
16                             UI/UX DESIGNER   
17  B2C Support - Creative and UI/UX Designer   
18                             UI/UX Designer   
19                                  UX Writer   

                                                 Cosine Similarity  \
                                 company                     title   
0   PT HZN Teknologi Indonesia (Horizon)  UI Designer - Internship   
1        Global Landscapes Forum (CIFOR)               UX Designer   
2                               Startups     Senior UI/UX Designer   
3                               Tokenomy     Senior UI/UX Designer   
4   PT HZN Teknologi Indonesia (Horizon)          UI / UX Designer   
5                             Tinkerlust            UI/UX DESIGNER   
6                            Cermati.com           UI/ UX Designer   
7               PT Mobile Sarana Sentosa     Senior UI/UX Designer   
8                       Whello Indonesia            UI/UX Designer   
9                               Pensieve            UI/UX Designer   
10     Mekari (PT. Mid Solusi Nusantara)     Senior UI/UX Designer   
11               PT Nusa Satu Inti Artha            UI/UX Designer   
12                               Devhaus            UI/UX Designer   
13                         1001malam.com            UI/UX Designer   
14         DOKU, PT NUSA SATU INTI ARTHA                 UX Writer   
15                             sribu.com          UX / UI Designer   
16                           Dipstrategy  UI/UX Designer (Officer)   
17                PT Asuransi Jiwa Astra            UI/UX Designer   
18                                 Aleph            UI/UX Designer   
19                                  NOBI            UI/UX Designer   

                                          
                                 company  
0   PT HZN Teknologi Indonesia (Horizon)  
1   PT HZN Teknologi Indonesia (Horizon)  
2                       Whello Indonesia  
3          DOKU, PT NUSA SATU INTI ARTHA  
4               PT Mobile Sarana Sentosa  
5                            Dipstrategy  
6                             Tinkerlust  
7                PT Nusa Satu Inti Artha  
8                               Dekoruma  
9    Nomura Research Institute Indonesia  
10                              Tokenomy  
11              NTT INDONESIA TECHNOLOGY  
12                                 Aleph  
13           Tera Logistik Internasional  
14                                  NOBI  
15                              Startups  
16          PT Majoo Teknologi Indonesia  
17     Mekari (PT. Mid Solusi Nusantara)  
18                  Bali Staff Solutions  
19                         1001malam.com

In [303]:
job_compare = pd.merge(df_compare['Minkowski'], df_compare['Cosine Similarity'], how = 'inner')
job_compare

title                               company
0                UX Designer  PT HZN Teknologi Indonesia (Horizon)
1           UX / UI Designer                              Startups
2      Senior UI/UX Designer                              Tokenomy
3   UI Designer - Internship  PT HZN Teknologi Indonesia (Horizon)
4            UI/ UX Designer                            Tinkerlust
5           UI / UX Designer              PT Mobile Sarana Sentosa
6      Senior UI/UX Designer                      Whello Indonesia
7             UI/UX Designer     Mekari (PT. Mid Solusi Nusantara)
8      Senior UI/UX Designer               PT Nusa Satu Inti Artha
9             UI/UX Designer                         1001malam.com
10     Senior UI/UX Designer         DOKU, PT NUSA SATU INTI ARTHA
11            UI/UX DESIGNER                           Dipstrategy
12            UI/UX Designer                                 Aleph
13                 UX Writer                                  NOBI

In [304]:
similarityPercentage.append((job_compare.shape[0]/df_compare.shape[0])*100)

In [305]:
print(resumeList)
print(similarityPercentage)

['Arbi Dwi.pdf', 'Dwiky Aprian.pdf', 'Fadjrian Gibran.pdf', 'Faisal_Rizki.pdf', 'Ferdian_Maulana.pdf', 'Hanif Khoirul.pdf', 'Ilyas Adiyasa.pdf', 'Rangga Syahrial.pdf', 'Triardy_S.pdf', 'uiux.pdf']
[30.0, 15.0, 40.0, 25.0, 65.0, 55.00000000000001, 10.0, 45.0, 40.0, 70.0]


In [306]:
modelSimilarity = pd.DataFrame(list(zip(resumeList, similarityPercentage)),columns =['Resume Name', 'Similarity'])
print(modelSimilarity)

           Resume Name  Similarity
0         Arbi Dwi.pdf        30.0
1     Dwiky Aprian.pdf        15.0
2  Fadjrian Gibran.pdf        40.0
3     Faisal_Rizki.pdf        25.0
4  Ferdian_Maulana.pdf        65.0
5    Hanif Khoirul.pdf        55.0
6    Ilyas Adiyasa.pdf        10.0
7  Rangga Syahrial.pdf        45.0
8        Triardy_S.pdf        40.0
9             uiux.pdf        70.0


In [307]:
print("Rata-rata similarity antara 2 metric diatas adalah: ", modelSimilarity["Similarity"].mean())

Rata-rata similarity antara 2 metric diatas adalah:  39.5
